In [1]:
import pandas as pd

import json
import glob
import os
import re

from shared import *

In [2]:
base = "../exp/"
paths = glob.glob(base + "data/*.json")
data = [json.load(open(path)) for path in paths]

In [3]:
queries = pd.DataFrame.from_records([{
    **algo,
    'graph': path_to_graph(run['args'][1]),
    'metric': run['live_weight_file'],
    'queries': run['queries'],
    'epsilon': run['epsilon'],
} for run in data for algo in run['algo_runs']])

In [4]:
queries['instance'] = queries['graph'].map({'OSM Europe': 'OSM Eur ', 'DIMACS Europe': 'DIMACS Eur ', 'OSM Germany': 'OSM Ger '}) + queries['metric'].map({'fake_traffic': 'Syn', 'heavy_traffic': 'Fri', 'lite_traffic': 'Tue'})

In [5]:
queries_sub = queries.query("~(graph == 'OSM Germany' & metric == 'fake_traffic') & epsilon == 0.5")
table = queries_sub.groupby(['queries', 'instance', 'algo']) \
    .mean()[['length_increase_percent', 'num_iterations', 'num_forbidden_paths', 'running_time_ms', 'failed']] \
    .unstack()

table['failed'] *= 100
table = table.round(1)
# for col in ['running_time_ms', 'num_forbidden_paths']:
#     table[col] = table[col].round(1)
# for col in ['length_increase_percent', 'num_iterations']:
#     table[col] = table[col].round(2)
table = table.rename(columns={ 'iterative_detour_blocking': 'IDB', 'iterative_path_blocking': 'IPB' })

# table = table.swaplevel(axis='columns').sort_index(axis='columns').reindex(columns=['length_increase_percent', 'num_iterations', 'num_forbidden_paths', 'running_time_ms', 'failed'], level=1)

output = table.to_latex()
output = output.replace('queries/1h', R"\multirow{4}{*}{\rotatebox[origin=c]{90}{1h}}")
output = output.replace('queries/4h', R"\addlinespace \multirow{4}{*}{\rotatebox[origin=c]{90}{4h}}")
output = output.replace('queries/uniform', R"\addlinespace \multirow{4}{*}{\rotatebox[origin=c]{90}{Random}}")
lines = output.split("\n")
lines = [R'\setlength{\tabcolsep}{4pt}'] + lines[:2] + [
    R" & & \multicolumn{2}{c}{Increase $[\%]$} & \multicolumn{2}{c}{Iterations} & \multicolumn{2}{c}{Blocked Paths} & \multicolumn{2}{c}{Time [ms]} & \multicolumn{2}{c}{Failed $[\%]$} \\",
    R"\cmidrule(l{3pt}r{3pt}){3-4} \cmidrule(l{3pt}r{3pt}){5-6} \cmidrule(l{3pt}r{3pt}){7-8} \cmidrule(l{3pt}r{3pt}){9-10} \cmidrule(l{3pt}r{3pt}){11-12}",
    R" & & IDB & IPB & IDB & IPB & IDB & IPB & IDB & IPB & IDB & IPB \\"
] + lines[5:]
output = add_latex_big_number_spaces("\n".join(lines) + "\n")

print(output)
table

\setlength{\tabcolsep}{4pt}
\begin{tabular}{llrrrrrrrrrr}
\toprule
 & & \multicolumn{2}{c}{Increase $[\%]$} & \multicolumn{2}{c}{Iterations} & \multicolumn{2}{c}{Blocked Paths} & \multicolumn{2}{c}{Time [ms]} & \multicolumn{2}{c}{Failed $[\%]$} \\
\cmidrule(l{3pt}r{3pt}){3-4} \cmidrule(l{3pt}r{3pt}){5-6} \cmidrule(l{3pt}r{3pt}){7-8} \cmidrule(l{3pt}r{3pt}){9-10} \cmidrule(l{3pt}r{3pt}){11-12}
 & & IDB & IPB & IDB & IPB & IDB & IPB & IDB & IPB & IDB & IPB \\
\midrule
\multirow{4}{*}{\rotatebox[origin=c]{90}{1h}} & DIMACS Eur Syn &                     1.8 &  0.7 &            3.5 &   20.6 &                 9.5 &   62.5 &            25.1 &    21.0 &    0.5 &   6.1 \\
                & OSM Eur Syn &                     0.2 &  0.1 &            1.7 &    6.5 &                 1.0 &   25.0 &            11.1 &     6.6 &    0.2 &   1.2 \\
                & OSM Ger Fri &                     0.8 &  0.1 &            3.8 &   26.1 &                13.9 &   64.4 &            91.7 &    43.2 &    0.6 &  

length_increase_percent      num_iterations  \
algo                                               IDB  IPB            IDB   
queries         instance                                                     
queries/1h      DIMACS Eur Syn                     1.8  0.7            3.5   
                OSM Eur Syn                        0.2  0.1            1.7   
                OSM Ger Fri                        0.8  0.1            3.8   
                OSM Ger Tue                        0.1  0.0            1.2   
queries/4h      DIMACS Eur Syn                     3.0  1.0           12.8   
                OSM Eur Syn                        0.2  0.2            2.5   
                OSM Ger Fri                        1.5  0.2           18.5   
                OSM Ger Tue                        0.2  0.0            2.0   
queries/uniform DIMACS Eur Syn                     2.8  1.0           27.6   
                OSM Eur Syn                        0.3  0.2           10.4   
                OSM Ger Fri                        1.3  0.2           14.3   
                OSM Ger Tue                        0.2  0.0            2.0   

                                      num_forbidden_paths         \
algo                              IPB                 IDB    IPB   
queries         instance                                           
queries/1h      DIMACS Eur Syn   20.6                 9.5   62.5   
                OSM Eur Syn       6.5                 1.0   25.0   
                OSM Ger Fri      26.1                13.9   64.4   
                OSM Ger Tue       8.3                 0.9   19.4   
queries/4h      DIMACS Eur Syn   97.8                47.3  410.4   
                OSM Eur Syn      25.9                 3.3  114.0   
                OSM Ger Fri     135.6               113.6  375.7   
                OSM Ger Tue      29.1                 6.9   75.2   
queries/uniform DIMACS Eur Syn  180.0               128.5  807.9   
                OSM Eur Syn     110.2                19.7  737.4   
                OSM Ger Fri     112.3                78.2  315.9   
                OSM Ger Tue      27.1                 7.2   73.1   

                               running_time_ms         failed        
algo                                       IDB     IPB    IDB   IPB  
queries         instance                                             
queries/1h      DIMACS Eur Syn            25.1    21.0    0.5   6.1  
                OSM Eur Syn               11.1     6.6    0.2   1.2  
                OSM Ger Fri               91.7    43.2    0.6   8.5  
                OSM Ger Tue                3.3     9.8    0.0   2.5  
queries/4h      DIMACS Eur Syn           200.5   251.1    2.2  34.1  
                OSM Eur Syn               14.0    58.3    0.3   5.3  
                OSM Ger Fri             1817.8   842.6    2.1  51.8  
                OSM Ger Tue               32.8    90.5    0.0   8.6  
queries/uniform DIMACS Eur Syn          4241.1  6096.4    4.1  67.6  
                OSM Eur Syn              548.5  6572.4    2.9  29.6  
                OSM Ger Fri             1523.4   741.5    1.3  42.0  
                OSM Ger Tue               39.6    88.2    0.0   8.4